**在 PostgreSQL 中，变量的声明需要在 PL/pgSQL 块或函数内部进行。**

In [ ]:
DO $$
DECLARE
  tbl varchar := 'aaa'; -- 声明并初始化一个 varchar 类型的变量
BEGIN
  -- 在这里可以使用变量 tbl 执行一些操作
  RAISE NOTICE 'Table name is: %', tbl; -- 例如，打印变量值
END
$$;

**支持 CREATE INDEX IF NOT EXISTS 来创建索引**

在 PostgreSQL 中，截至我最后更新的知识（2023年），并没有直接的语法支持 CREATE INDEX IF NOT EXISTS 来创建索引，如果索引不存在的话。这与 ALTER TABLE IF NOT EXISTS 的情况类似，PostgreSQL 没有提供 IF NOT EXISTS 子句用于索引创建。

你遇到的错误是因为 PostgreSQL 无法识别 IF NOT EXISTS 这部分语法。为了实现相同的功能，你可以使用以下方法来避免创建已存在的索引：
方法 1: 显式检查索引是否存在

你可以使用 pg_indexes 系统表来检查索引是否存在，然后根据检查结果决定是否创建索引：

DO $$
BEGIN
    IF NOT EXISTS (
        SELECT 1 FROM pg_indexes 
        WHERE schemaname = 'public' AND indexname = 'pgr_source_idx'
    ) THEN
        CREATE INDEX pgr_source_idx ON public.aaa(pgr_source);
    END IF;
END
$$;

方法 2: 使用 PL/pgSQL 块

使用 PL/pgSQL 块来编写一个程序，该程序首先检查索引是否存在，如果不存在则创建索引：

BEGIN
    -- 检查索引 'pgr_source_idx' 是否存在
    IF NOT EXISTS (
        SELECT 1 FROM pg_indexes 
        WHERE schemaname = 'public' AND indexname = 'pgr_source_idx'
    ) THEN
        -- 如果不存在，创建索引
        CREATE INDEX pgr_source_idx ON public.aaa(pgr_source);
    END IF;
END;

方法 3: 使用 pg_constraint 检查唯一索引

如果你正在创建的是一个唯一索引，你也可以通过检查 pg_constraint 表来实现：

DO $$
DECLARE
    index_exists BOOLEAN := FALSE;
BEGIN
    SELECT conname INTO index_exists
    FROM pg_constraint 
    WHERE conname = 'pgr_source_idx';
    
    IF NOT index_exists THEN
        CREATE UNIQUE INDEX pgr_source_idx ON public.aaa(pgr_source);
    END IF;
END
$$;

请注意，这些示例中的 pg_indexes 表包含了所有索引的信息，包括普通索引和唯一索引。pg_constraint 表则包含了唯一索引和外键约束的信息。确保在执行这些操作之前备份数据库，以防万一。

**ALTER TABLE 语句本身并不支持 IF NOT EXISTS 子句来检查列是否存在**

PostgreSQL 的 ALTER TABLE 语句用于修改已存在的表结构，例如添加、删除列或修改列的属性。但是，ALTER TABLE 语句本身并不支持 IF NOT EXISTS 子句来检查列是否存在。这意味着你不能直接在 ALTER TABLE 语句中使用 IF NOT EXISTS 来避免添加已存在的列。

错误信息指出 PostgreSQL 在解析你的 SQL 语句时遇到了语法错误。具体来说，是在尝试执行 IF NOT EXISTS 检查时出现问题。

要解决这个问题，你可以使用以下方法之一：
方法 1: 先检查列是否存在，然后添加

你可以先查询 information_schema.columns 表来检查列是否存在，然后再执行 ALTER TABLE 语句：

DO $$
BEGIN
    IF NOT EXISTS (
        SELECT 1
        FROM information_schema.columns 
        WHERE 
            table_name=CURRENT_QUERY() AND 
            column_name='pgr_source'
    ) THEN
        ALTER TABLE public.aaa ADD COLUMN pgr_source integer;
    END IF;
END
$$;

方法 2: 使用 PL/pgSQL 块

使用 PL/pgSQL 语言块来编写一个程序，该程序首先检查列是否存在，如果不存在则添加列：

BEGIN
    -- 检查列 'pgr_source' 是否存在
    IF NOT EXISTS (
        SELECT 1
        FROM information_schema.columns 
        WHERE 
            table_schema = 'public' AND 
            table_name   = 'aaa' AND 
            column_name  = 'pgr_source'
    ) THEN
        -- 如果不存在，添加列
        ALTER TABLE public.aaa ADD COLUMN pgr_source integer;
    END IF;
END;

方法 3: 使用系统表

在某些情况下，你也可以直接使用 PostgreSQL 的系统表 pg_attribute 来检查列是否存在：

DO $$
DECLARE
    column_exists INTEGER;
BEGIN
    SELECT COUNT(1) INTO column_exists
    FROM pg_attribute
    WHERE attrelid = 'public.aaa'::regclass AND attname = 'pgr_source';

    IF column_exists = 0 THEN
        ALTER TABLE public.aaa ADD COLUMN pgr_source integer;
    END IF;
END
$$;

这些方法可以帮助你避免因尝试添加已存在的列而导致的错误。选择适合你需求的方法，并确保在执行这些操作之前备份数据库，以防万一。